# Linear Optimization With Demand Uncertainty Conditions 


1. update temp at each actual sales realised and then run whole notebook until the current week 
2. Repeat until finished 
3. Enjoy the results 

In [816]:
import pandas as pd
import numpy as np

# # simulations
# simulations = pd.read_csv('/Users/arham/Downloads/RM/Group Assignment 1/Simulation.csv')

# simulations dataframe empty with columns week, price, sales
simulations = pd.DataFrame(columns=['Week', 'Sales'])
Tracker = pd.DataFrame(columns=['Week', 'Price', 'Sales', 'Forecast', 'Gap'])


# add week starting 1 to 15
simulations['Week'] = np.arange(1, 16)
Tracker['Week'] = np.arange(1, 16)


# Tempo = [97, 142, 136,149,110,84,94,104,194] # 2.3% 
# Tempo = [78,89,65,76,76,148,159,180,13,193, 175,153,153,180,225] # Market Crash 5%
# Tempo = [112,85,136,147,113,184,132,204,163,170,210,140] # 4%
Tempo = [80,71,136,84,139,27,174,141,211,161,159,101,175,130] #1.1% # please iterate each element one by one, add them one by one instead of running whole list at one go

# add values to simulations 'Sales' column, if values are less then 0
# then set them to 0
simulations['Sales'] = 0 
for j in range(len(Tempo)):
    simulations['Sales'][j] = Tempo[j] 

Tracker['Sales'] = simulations['Sales']

simulations
Tracker


,Week,Price,Sales,Forecast,Gap
0,1,NaN,80,NaN,NaN
1,2,NaN,71,NaN,NaN
2,3,NaN,136,NaN,NaN
3,4,NaN,84,NaN,NaN
4,5,NaN,139,NaN,NaN
5,6,NaN,27,NaN,NaN
6,7,NaN,174,NaN,NaN
7,8,NaN,141,NaN,NaN
8,9,NaN,211,NaN,NaN
9,10,NaN,161,NaN,NaN


In [817]:
import pandas as pd
import matplotlib.pyplot as plt

# Data
data = {
    "Week": range(2, 16),
    "Weekly Increment (60)": [13.50, 9.78, 21.34, 1.29, -7.95, 24.52, -11.26, -25.29, -19.99, -20.62, -20.72, -20.67, -20.68, -20.68],
    "Weekly Increment (54)": [21.12, 25.62, 23.24, 16.61, 25.29, 14.86, 48.10, 11.32, -16.50, 39.66, -13.76, 3.60, -17.64, 45.30],
    "Weekly Increment (48)": [5.42, 5.45, 5.37, 5.13, 5.49, 5.92, 7.91, 1.59, 2.00, 16.00, -4.34, 24.70, -26.55, 46.35],
    "Weekly Increment (36)": [5.58, 5.59, 5.56, 5.51, 5.38, 6.11, 6.21, 4.55, 4.09, 2.00, 30.00, 10.31, 20.21, 48.38]
}

# Create a DataFrame
Demand_Increments = pd.DataFrame(data)
# add 1 in each element of the dataframe and divide by 100
Demand_Increments = Demand_Increments.div(100).add(1)

Uplift_predictions = Demand_Increments

In [818]:
Demand_Increments

,Week,Weekly Increment (60),Weekly Increment (54),Weekly Increment (48),Weekly Increment (36)
0,1.02,1.1350,1.2112,1.0542,1.0558
1,1.03,1.0978,1.2562,1.0545,1.0559
2,1.04,1.2134,1.2324,1.0537,1.0556
3,1.05,1.0129,1.1661,1.0513,1.0551
4,1.06,0.9205,1.2529,1.0549,1.0538
5,1.07,1.2452,1.1486,1.0592,1.0611
6,1.08,0.8874,1.4810,1.0791,1.0621
7,1.09,0.7471,1.1132,1.0159,1.0455
8,1.10,0.8001,0.8350,1.0200,1.0409
9,1.11,0.7938,1.3966,1.1600,1.0200


In [819]:
# # runs where week 14 inventory is 0
# NonStockOutCases = simulations[(simulations['Remaining Inventory'] != 0) & (simulations['Week'] == 14)]['Run'].unique()

# Sample = simulations[simulations['Run'] == 90]

# Sample = Sample[['Week','Sales']]

Sample = simulations[['Week','Sales']]

For the model to be realistic we cannot see the future, thus we will have a new optimization problem every week to incorporate historical and present condition of market.


We Will use the following variables to track weekly pricing decisions, sales prediction, sales, inventory 

In [820]:
# pricing Decisions 
Price_decisions = []
Actual_Sales = [] 
Predictions = []
Weekly_Inventory = []


Price_List = [60, 54, 48, 36]
Sales_Deltas = Uplift_predictions.drop(columns=['Week'])



Let's start with the Week 1: 
The only thing we realise in week one is week one sales with no decisions made


In [821]:
Price_decisions.append(60)
Actual_Sales.append(Sample['Sales'].iloc[0])
Predictions.append(-1)
Weekly_Inventory.append(2000)


# print variables
print('Week 1')
print('-----------------')
print('Price Decision:', Price_decisions[-1])
print('Actual Sales:', Actual_Sales[-1])
print('Sales Prediction:', Predictions[-1])
print('Starting Inventory:', Weekly_Inventory[-1])

Tracker['Price'] = Price_decisions[-1]
Tracker['Forecast'] = Predictions[-1]


Week 1
-----------------
Price Decision: 60
Actual Sales: 80
Sales Prediction: -1
Starting Inventory: 2000


Average weekly demand increment is calculated for each price based on given 'Sales-Data'


In [822]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd


In [823]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][0]
Weekly_Inventory.append(new_inventory) #enter value from game
Weekly_Inventory

[2000, 1920]

### Week 2 - Optimization Problem

In [824]:
k = 1 # decision 1

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]


Uplift_predictions = Sales_Deltas.iloc[k-1].values


# Uplift_predictions = [0.6, 1.05, 1.50, 1.60]

# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))




m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))



# # print variables
# print('Week 2')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# print('Sales Prediction:', Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])
# Check if the optimization was successful
if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x1ca94b0d
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros
Found heuristic solution: objective 5448.0000000
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 5448 -0 

Op

In [825]:
Price_decisions

[60, 60.0]

In [826]:
jumps = 1.3
lower_threshold = 63
upper_threshold = 100

## Week 3

In [827]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0



# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

# Check if the optimization was successful
if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0xb739bb4a
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros
Found heuristic solution: objective 4676.6280000
Variable types: 0 continuous, 4 integer (4 binary)
Found heuristic solution: objective 4984.2000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 avai

## Week 4

In [828]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

## Constraints that compare previous week price, and sets the markdown upgrade ratios if for all options
if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint: if the penultimate sales were greater than 100, then we should not change the price
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

      
# variable for sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton 
# sales predicted if there is inventory is equal to previous week sales multiplied by uplift percentage for chosen price
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
# if there is not enough inventory then sales_predicted is equal to avialable inventory
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# constraint penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x689a78fb
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 9050.75 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.050745600000e+03, best bound 9.050745600000e+03, gap 0.0000%
Week 4
-----------------
Price Decision: 54.0
Sale

## Week 5

In [829]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

      
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])
if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0xfb0ed1e1
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Found heuristic solution: objective 3931.2000000
Variable types: 0 continuous, 3 integer (3 binary)
Found heuristic solution: objective 5289.4296000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 avai

Week 6

In [830]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

      
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x24de9861
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 9404.27 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.404267400000e+03, best bound 9.404267400000e+03, gap 0.0000%
Week 6
-----------------
Price Decision: 54.0
Sale

Week 7

In [831]:
Penultimate_Sales = Sample['Sales'].iloc[k-1]
Penultimate_Sales
Sample['Sales']


0      80
1      71
2     136
3      84
4     139
5      27
6     174
7     141
8     211
9     161
10    159
11    101
12    175
13      0
14      0
Name: Sales, dtype: int64

In [832]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

      
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))


m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0xc511102e
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Found heuristic solution: objective 1263.6000000
Variable types: 0 continuous, 2 integer (2 binary)
Found heuristic solution: objective 1684.8000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 1684.8 1263.6 -0 

Optimal solutio

Week 8

In [833]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

     
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x2382e653
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 9012.64 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.012643200000e+03, best bound 9.012643200000e+03, gap 0.0000%
Week 8
-----------------
Price Decision: 48.0
Sale

Week 9

In [834]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

    
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x4663155d
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 6875.61 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.875611200000e+03, best bound 6.875611200000e+03, gap 0.0000%
Week 9
-----------------
Price Decision: 48.0
Sale

Week 10

In [835]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

      
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x3766857b
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 10330.6 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.033056000000e+04, best bound 1.033056000000e+04, gap 0.0000%
Week 10
-----------------
Price Decision: 48.0
Sal

Week 11

In [836]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

              
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x338e4ca7
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 8964.48 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.964480000000e+03, best bound 8.964480000000e+03, gap 0.0000%
Week 11
-----------------
Price Decision: 48.0
Sal

Week 12

In [837]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

              
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x93b14faa
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 7300.77 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.300771200000e+03, best bound 7.300771200000e+03, gap 0.0000%
Week 12
-----------------
Price Decision: 48.0
Sal

Week 13

In [838]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

              
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x28dc965c
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 6045.46 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.045456000000e+03, best bound 6.045456000000e+03, gap 0.0000%
Week 13
-----------------
Price Decision: 48.0
Sal

Week 14

In [839]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

       
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x203694a8
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 6169.8 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.169800000000e+03, best bound 6.169800000000e+03, gap 0.0000%
Week 14
-----------------
Price Decision: 48.0
Sale

Week 15

In [840]:
new_inventory = Weekly_Inventory[-1] - simulations['Sales'][k-1]
Weekly_Inventory.append(new_inventory) #enter value from game

# Model code gurobi
m = gp.Model("Price Optimization")

## Decision Variable

# X[i][k] : Decision of what price to set for product i in week k. A binary variable, 
# where i = 0, 1 ,2, 4 representing price 60, 56, 48, 36 respectively, and k belongs to integers in 0 to 13
# gurobi variable X
X = m.addVars(4, vtype=GRB.BINARY, name=[f"Decision of Price {i} in Week {k+1}" for i in range(4)])

Penultimate_Sales = Sample['Sales'].iloc[k-1]

Uplift_predictions = Sales_Deltas.iloc[k-1].values.tolist()

if Price_decisions[-1]/6 == 6:
    temporary = 0
elif Price_decisions[-1]/6 == 8: 
    Uplift_predictions[3] =jumps
elif Price_decisions[-1]/6 == 9:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
elif Price_decisions[-1]/6 == 10:
    Uplift_predictions[3] = jumps
    Uplift_predictions[2] = jumps
    Uplift_predictions[1] = jumps

## Price Gap Threshold Constraint: If penultimate sales was less than 70, then we should decrease price
if Penultimate_Sales <= lower_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[1] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[0] = 0

## Good Market Constraint
if Penultimate_Sales >= upper_threshold:
    if Price_decisions[-1]/6 == 6:
        temporary = 0
    elif Price_decisions[-1]/6 == 8: 
        Uplift_predictions[3] =0
    elif Price_decisions[-1]/6 == 9:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
    elif Price_decisions[-1]/6 == 10:
        Uplift_predictions[3] = 0
        Uplift_predictions[2] = 0
        Uplift_predictions[1] = 0

               
# sales predictions for each price
Sales_Predictions = m.addVars(4, vtype=GRB.CONTINUOUS, name=[f"Sales Prediction for Price {i} in Week {k+1}" for i in range(4)])


## Set Objective to maximize revenue for week 2
m.setObjective(gp.quicksum(X[i]*Price_List[i]*Sales_Predictions[i] for i in range(4)), GRB.MAXIMIZE)

## Constraints

# Constraint 1: The sum of all prices decisions set in week 2 should be equal to 1
m.addConstr(gp.quicksum(X[i] for i in range(4)) == 1)

# constraint 2,3: Definition of sales prediciton and Predicted SAles should be less than available inventory
m.addConstrs((Sales_Predictions[i] <= Penultimate_Sales * Uplift_predictions[i] for i in range(4)), name = 'Sales Prediction')
m.addConstr(gp.quicksum(Sales_Predictions[i] for i in range(4)) <= Weekly_Inventory[-1])

# penultimate price decision should be greater than current price
m.addConstr((gp.quicksum(X[i]*Price_List[i] for i in range(4)) <= Price_decisions[-1]))



m.optimize()
# # append variables to lists
# Price_decisions.append(gp.quicksum([X[i].x*Price_List[i] for i in range(4)]))
# Predictions.append([Sales_Predictions[i].x for i in range(4)])
# Actual_Sales.append(Sample['Sales'].iloc[k])
# Weekly_Inventory.append(Weekly_Inventory[-1] - min(sum([Sales_Predictions[i].x for i in range(4)]), Weekly_Inventory[-1]))


# # print variables
# print(f'Week {k+1}')
# print('-----------------')
# print('Price Decision:', Price_decisions[-1])
# print('Actual Sales:', Actual_Sales[-1])
# # print('Sales Prediction:', Sales_Predictions[-1])
# print('Weekly Inventory:', Weekly_Inventory[-1])

if m.status == GRB.OPTIMAL:
    # Appending the total price decision
    total_price_decision = sum(X[i].x * Price_List[i] for i in range(4))
    Price_decisions.append(total_price_decision)

    # Appending sales predictions for each price
    sales_predictions = [Sales_Predictions[i].x for i in range(4)]
    Predictions.append(sales_predictions)

    # Appending actual sales from the sample data
    actual_sales = Sample['Sales'].iloc[k]
    Actual_Sales.append(actual_sales)


    # Printing results for review
    print('Week', k+1)
    print('-----------------')
    print('Price Decision:', total_price_decision)
    print('Sales Prediction:', sales_predictions)
    print('Actual Sales:', actual_sales)
    print('Starting Inventory:', Weekly_Inventory[-1])
else:
    print("Optimization did not reach an optimal solution.")



k+=1

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x47b31f2e
Model has 4 quadratic objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 7 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
Week 15
--------------